<a href="https://colab.research.google.com/github/twinstae/tripReviewAnalysisSystem/blob/master/%ED%81%AC%EB%A1%A4%EB%9F%AC-%EC%A0%84%EC%B2%98%EB%A6%AC/TRAS_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q selenium
!pip install -q bs4
!pip install -q pandas

     |████████████████████████████████| 911kB 2.8MB/s eta 0:00:01


In [1]:
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import time
import re
import sys
from PyQt5.QtWidgets import *
from PyQt5 import uic

In [2]:
def data_to_csv(attraction_sort, start, end, selected_page):
    git_address = 'C:/Users/Jeong/'
    
    attraction_sort.to_csv(git_address + str((selected_page-1)*30 + start)+'-'+ str((selected_page-1)*30 + end) + '.csv', mode = 'w')

In [25]:
def duration_crawls():
       
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    attraction_header = soup.find(class_ = 'attractions-attraction-review-atf-AttractionReviewATFLayout__atf_left_column--1ogzd')
    suggesting = attraction_header.find_all(class_ = 'attractions-attraction-detail-about-card-AboutSection__sectionWrapper--3PMQg')[0]
    # 원하는 정보가 담겨있는 전체 페이지 선택
    
    try:
        full_duration = suggesting.text
        start_duration = full_duration.find(':') + 1
        full_duration = full_duration[start_duration:]
    except:
        suggest_duration = float('nan')

    duration = [attraction_header.find('h1', class_='ui_header h1').text, full_duration]

    return duration

In [4]:
def reviews(sample_size):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 표본 리뷰의 숫자자.
    맥락 : 리뷰를 추출할 즐길거리 페이지에 있다.
    출력 : 한 즐길거리에 대한 리뷰를 모아놓은 데이터 프레임

    각 리뷰에서 새로운 항목을 추출하고 싶다면, columns와 for문을 모두 수정해야한다.
    '''
    reviews = []
    columns = ['star_point','title','text','Date of experience']
    #,'reviewer_address'
        
        
    #리뷰 크기를 텍스트로 찾아와, 콤마를 제거하여, 정수형으로 저장.
    review_size_text = driver.find_element_by_class_name('attractions-community-content-TabBarContent__tabCount--2hTdj').text
    review_size = int(re.sub(',','', str(review_size_text)))

    n = 5
    count = 0
    while n < sample_size:
        try:
            foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
            show_more = foot.find_element_by_tag_name('button')  
            driver.execute_script("arguments[0].click();", show_more)
            n += 5
        except:
            if count > 3:
                break
            count +=1
            time.sleep(2)
                
    review_cards = driver.find_elements_by_class_name('location-review-card-Card__ui_card--2Mri0')

    for review_card in review_cards:
        try:
            read_more = review_card.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
            read_more.click()
        except:
            print('error')
            continue
    
    # 드라이버 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')
    
    for review_element in review_elements:
        try:
            review = [
                int(review_element.find('span',class_='ui_bubble_rating')['class'][1][-2:]) / 10,
                review_element.find('a', class_='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT').find('span').find('span').text,
                review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').find('span').text,
                review_element.find('span', class_='location-review-review-list-parts-EventDate__event_date--1epHa').contents[1]
                #,review_element.find('span', class_='social-member-common-MemberHometown__hometown--3kM9S').contents[1]
            ]
        except IndexError:
            #아무도 helpful이나 vote를 누르지 않은 에러, 무시하고 계속
            continue
        except AttributeError:
            #주소를 공개하지 않는 에러, 무시하고 계속
            continue
        reviews.append(review)

    return pd.DataFrame(reviews, columns = columns)

In [5]:
def crawl_reviews():
    
    columns = ['title','text']
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        member_review = driver.find_element_by_class_name('pageNumbers')
        group_size = int(member_review.find_elements_by_tag_name('a')[-1].text)
        m_review = 1
        
    except:
        group_size = 2
        m_review = 2
        
    if group_size >= 60 :
        group_count = 60
    else :
        group_count = group_size
        
    review_count = 1
    reviews = []
    
    while review_count < group_count:
        
        review_cards = driver.find_elements_by_class_name('location-review-card-Card__ui_card--2Mri0')
        time.sleep(3)
        try:
            read_more = review_cards[0].find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
            driver.execute_script("arguments[0].click();", read_more)
        except ElementNotInteractableException:
            print("I already Click the read more buttons!")
        except ElementClickInterceptedException:
            print("I can't click the read more buttons!")
        except StaleElementReferenceException:
            print("I can't click the read more buttons!")
        except:
            break
            
            
        time.sleep(2)
        
        # 드라이버 
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')

        for review_element in review_elements:
            try:
                review = [
                    int(review_element.find('span',class_='ui_bubble_rating')['class'][1][-2:]) / 10,
                    review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').find('span').text
                ]
            except IndexError:
                #아무도 helpful이나 vote를 누르지 않은 에러, 무시하고 계속
                continue
            except AttributeError:
                #주소를 공개하지 않는 에러, 무시하고 계속
                continue
            reviews.append(review)

        review_count += 1
        
        
        
        str_review_count = str(review_count)
        if m_review == 1:
            try:
                member_next = member_review.find_element_by_link_text(str_review_count)
                driver.execute_script("arguments[0].click();", member_next)
            except StaleElementReferenceException:
                print("No reivew page!")
                break
            except NoSuchElementException:
                print("No reivew page!")
                break
        else:
            print ("No reivew page number")
            break
        time.sleep(3)
    df = pd.DataFrame(reviews, columns = columns)

    return (df)

In [6]:
def find_address():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contact = soup.find_all(class_ = 'attractions-contact-card-ContactCard__contactRow--3Ih6v')
    address_block = contact[0]
    address_spans = address_block.contents
    address_span = address_spans[1]
    address = address_span.text
    return address

"""data_test = [names, location]

df_list = pd.DataFrame(data_test, columns = columns)
df_list.to_csv('crawler test.csv')
"""

"data_test = [names, location]\n\ndf_list = pd.DataFrame(data_test, columns = columns)\ndf_list.to_csv('crawler test.csv')\n"

In [7]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [8]:
#해당 여행지의 페이지로 이동한 뒤에 호출
def crawl_attraction_by_types():
    '''
    현재 페이지에 있는 리뷰들을 여행자 타입별로 데이터프레임의 딕셔너리로 만들어 반환
    '''
    
    review_panel = driver.find_element_by_class_name('location-review-review-list-parts-ReviewFilters__filters_wrap--y1t86')    
    process1 = review_panel.find_elements_by_class_name('is-shown-at-tablet')
    pp.pprint([we.text for we in process1])
    checkbox_list = process1[0].find_elements_by_tag_name('label')
    pp.pprint([we.text for we in checkbox_list])    
    
    checkbox_dict = {checkbox.text: checkbox for checkbox in checkbox_list}
    print("I found the checkboxes!")
    pp.pprint(checkbox_dict)
    attraction_dict = {}
    for traveler_type, check in checkbox_dict.items():
        print(traveler_type)
        
        check.click()
        time.sleep(2)
        data = crawl_reviews()
        attraction_dict[traveler_type] = data
        check.click()
    return attraction_dict

In [22]:
def load_attraction(link):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 어떤 즐길거리 번호 int, 이름 string 예: "Gyeongbokgung Palace"
    출력 : 어트랙션의 리뷰 모음. data.frame
    '''
    #새 창을 열고 링크로 간다.
    # 슬립...매우매우중요이거없으면무조건에러남.이거하나로3시간머리싸맸음.
    
    time.sleep(2)
    original_window = driver.current_window_handle
    driver.execute_script("arguments[0].click();", link)
    
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
    
    #즐길거리 별로 샘플사이즈를 어떻게 정할 것인지?    
    result = duration_crawls()

    #창을 닫고 리스트 창으로 돌아간다
    driver.close();
    driver.switch_to.window(original_window)
    driver.current_window_handle;
    
    return result
    #result.head()

In [18]:
def load_page(start, end, selected_page):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 크롤링할 페이지 내 시작 번호와 끝 번호
    출력 : 페이지의 리뷰들을 csv파일로 출력
    '''

    #모든 여행지의 리스트를 만든다
    is_first = True
    print("I am in the load_page")
    try:
        main_page = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__container--3eHZU')    
        attraction_elements = main_page.find_elements_by_class_name("attractions-attraction-overview-pois-PoiInfo__info--239IR")
        is_first = True
    except NoSuchElementException :
        main_page = driver.find_element_by_class_name('attractions-attraction-filtered-common-attraction-filtered-common__listingsContainer--2cOBG')
        attraction_elements = main_page.find_elements_by_class_name('_1MKm6PFo')
        is_first = False
    print("I found the elements!")

    attraction_names = []
    for attraction_element in attraction_elements[start:end]:      
        if is_first:
            link = attraction_element.find_element_by_tag_name('h3')
            attraction_name = link.text[link.text.find('.')+2 :]
        else:
            link = attraction_element.find_element_by_tag_name('h2')
            attraction_name = link.text
        attraction_names.append((attraction_name, link))  
    print("I found the attraction_names")
    
    columns = ['name','suggest_time']
    attraction_list = []
    for attraction_name, link in attraction_names:
        if len(attraction_list) < end:
            attraction_list.append(load_attraction(link))
            if len(attraction_list) % 5 == 0:
                data_to_csv(DataFrame(attraction_list, columns = columns), start, end, selected_page)
                print("저장했어요!" + str((selected_page-1)*30 + start)+'-'+ str((selected_page-1)*30 + end))
                
        else:
            data_to_csv(attraction_dict)
            print("저장했어요!")
            break

In [11]:
attraction_dict ={'taeheegung': 'name'}
value_type = type(list(attraction_dict.values())[0])
value_type == type('s')

True

In [12]:
def func_is_first_page(driver):
    try:
        time.sleep(1)
        show = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-')
        driver.execute_script("arguments[0].click();", show)
        is_first_page = True
        print("it is first page!")
    except NoSuchElementException:
        print('continue!')
        is_first_page = False
    except:
        print("exception")

In [13]:
def get_start_page(soup, start, end):
    result_page = 1
    try:
        page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
    except:
        page_numbers = driver.find_element_by_class_name('attractions-attraction-filtered-common-FilteredPagination__pagination--15y40')
        print('버튼들을 찾았다!')
        
    if start // 30 > 0:
        if start // 30 > 6:
            target_num = 6
        else:
            target_num = (start // 30) + 1
        target_page = page_numbers.find_element_by_link_text(str(target_num))
        time.sleep(1)
        driver.execute_script("arguments[0].click();", target_page)
        time.sleep(1)
        result_page = target_num
        try:
            page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
        except NoSuchElementException:
            page_numbers = driver.find_element_by_class_name('attractions-attraction-filtered-common-FilteredPagination__pagination--15y40')
        is_first_page = False
    else:
        page_numbers_soup = soup.find("div", class_='attractions-attraction-overview-main-Pagination__container--PUXGq') 
        result_page = page_numbers_soup.find_all("span")[1].text
        page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
        
    return int(result_page), page_numbers


In [14]:
def crawl_pages(is_first_page, start, end):
    keep_going = True    
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    print("I made a soup")
    
    selected_page, page_numbers = get_start_page(soup, start, end)

    while keep_going:
        # 현재 페이지 번호 알아내기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print("I made a soup")
            
                # 시작점이 뒷 페이지면 지나간다
        if start <= selected_page * 30:
            strat_in_page = start-(selected_page-1)*30
            if strat_in_page < 0:
                strat_in_page = 0
            assert strat_in_page <= 30
            
            end_in_page = end-(selected_page-1)*30
            if end_in_page > 30:
                end_in_page = 30                
            assert 0 <= end_in_page
            if is_first_page:
                is_first_page = False
                
            load_page(strat_in_page, end_in_page, selected_page)
        
        # 끝나는 지점을 지나갔으면 루프를 끝낸다.
        if (selected_page - 1) * 30 >= end:
            keep_going = False
            break
            
        print("I find the current page")

        # 다음 페이지로 넘어가기
        time.sleep(1)
        selected_page += 1
        next_page = page_numbers.find_element_by_link_text(str(selected_page))
        driver.execute_script("arguments[0].click();", next_page)
        

In [15]:
def crawl_function(start, end):
    '''
    입력 : 크롤링할 즐길거리의 시작 번호와 끝 번호
    출력 : 해당 구간의 즐길거리 리뷰들csv로 출력
    '''    
        
    #처음 한 번만 see more을 누른다.
    is_first_page = func_is_first_page(driver)
    
    time.sleep(1)
    crawl_pages(is_first_page, start, end)

In [23]:
options = webdriver.ChromeOptions()
options.add_argument("lang=en-CA")
options.add_argument('window-size=1920x1080')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver_address = r'C:\Users\Jeong\chromedriver.exe'
driver = webdriver.Chrome(driver_address, chrome_options=options)
wait = WebDriverWait(driver, timeout=2)
driver.implicitly_wait(5)
# tripadvisor에 접근한다.
driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html')
#driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-oa30-Seoul.html')
#원래 있던 리스트 창을 저장해둔다
original_window = driver.current_window_handle






C:\Users\Jeong\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [24]:
crawl_function(start= 0, end= 30)

it is first page!
I made a soup
I made a soup
I am in the load_page
I found the elements!
I found the attraction_names


IndexError: list index out of range